<font size=6> arraylize

In [1]:
import numpy as np
def convert_sequences_to_array(sequences):
    '''
    inputs: sequence of nucleotides represented as a string composed of A, C, G, T
    outputs: a list of numpy array representations of a sequence with:
             A = [1, 0, 0, 0]
             C = [0, 1, 0, 0]
             G = [0, 0, 1, 0]
             T = [0, 0, 0, 1]
             
    '''

    nucleotide_array_dict = {'A': [1, 0, 0, 0],
                             'C': [0, 1, 0, 0],
                             'G': [0, 0, 1, 0],
                             'T': [0, 0, 0, 1],
                             'N': [0.25,0.25,0.25,0.25],
                             'R':[0.5,0,0.5,0],
                             'Y':[0,0.5,0,0.5],
                             'M':[0.5,0.5,0,0],
                             'K':[0,0,0.5,0.5],
                             'S':[0,0.5,0.5,0],
                             'W':[0.5,0,0,0.5],
                             'B':[0,0.33,0.33,0.33],
                             'D':[0.33,0,0.33,0.33],
                             'H':[0.33,0.33,0,0.33],
                             'V':[0.33,0.33,0.33,0]}
    
    sequence_array_list = []
    for seq in sequences:
        seq_array = []
        for nuc in seq:
            seq_array.append(nucleotide_array_dict[nuc])
        seq_array = np.array(seq_array) 
        sequence_array_list.append(seq_array)
    return sequence_array_list

<font size=6> Get TATA_TSS,nonTATA_TSS,TATA_nonTSS,nonTATA_nonTSS

In [21]:
import random
def get_TSS_nonTSS(gff_name,fasta_name):   
    rdata=[]
    comment=[]
    delnum=0
    with open(gff_name,"r") as f:
        rdata=f.readlines()
    for i in range(9):
        del(rdata[0])
    for i in range(len(rdata)):
        if rdata[i][0]=="#":
            comment.append(i)
        rdata[i]=rdata[i].split()
    for i in range(len(comment)):
        del(rdata[comment[i]-delnum])
        delnum+=1
    isgene=[]
    for i in range(len(rdata)):
        if rdata[i][2]=="gene":
            isgene.append(rdata[i])
    typelist=[]
    for i in range(len(rdata)):
        if rdata[i][2] not in typelist:
            typelist.append(rdata[i][2])
    genetrandp=[]
    genetrandn=[]
    genetrandun=[]
    for i in range (len(isgene)):
        if isgene[i][6]=="+":
            genetrandp.append([isgene[i][0],isgene[i][3],isgene[i][4],isgene[i][6]])
        elif isgene[i][6]=="-":
            genetrandn.append([isgene[i][0],isgene[i][3],isgene[i][4],isgene[i][6]])
        else:
            genetrandun.append([isgene[i][0],isgene[i][3],isgene[i][4],isgene[i][6]])

    TSSlocation=[]

    for i in range(len(genetrandp)):
        TSSlocation.append([genetrandp[i][0],int(genetrandp[i][1])-50,int(genetrandp[i][1])+200])
    rfasta=[]
    fasta2=[]
    genename=[]
    string=""
    with open(fasta_name,"r") as f:
        rfasta=f.readlines()
    for i in range(len(rfasta)):
        if rfasta[i][0]==">":
            fasta2.append(string)
            string=""
            genename.append(rfasta[i])
        else:
            string+=rfasta[i].rstrip()
    del(fasta2[0])
    del(genename[-1]) 
    for i in range(len(genename)):
        genename[i]=genename[i].split()[0].lstrip(">")
    genefasta={}
    for i in range(len(genename)):
        genefasta.update({genename[i]:fasta2[i]})
    TSSfasta=[]
    for i in range(len(TSSlocation)):
        if (TSSlocation[i][0] in genefasta.keys()):
            TSSfasta.append(genefasta[TSSlocation[i][0]][TSSlocation[i][1]:TSSlocation[i][2]].upper())
    n=0        
    while(n<len(TSSfasta)):
        while(len(TSSfasta[n])!=250):
            del(TSSfasta[n])
        n+=1
    nonTSSstart=[]
    nonTSSname=[]
    for i in range(50000):
        temp=random.randint(0,len(genetrandp)-1)
        if int(genetrandp[temp][2])-250>int(genetrandp[temp][1]):
            nonTSSname.append(temp)
            nonTSSstart.append(random.randint(int(genetrandp[temp][1]),int(genetrandp[temp][2])-250))
    nonTSSfasta=[]           
    for i in range(len(nonTSSname)):
        if  genetrandp[nonTSSname[i]][0] in genefasta.keys():
            name=genetrandp[nonTSSname[i]][0]
            nonTSSfasta.append(genefasta[name][nonTSSstart[i]:nonTSSstart[i]+250].upper())
    TATA_TSSfasta=[]
    TATA_nonTSSfasta=[]
    for i in range(len(TSSfasta)):
        for j in range(45):
            if TSSfasta[i][j:j+4]=="TATA":
                TATA_TSSfasta.append(TSSfasta[i])
    for i in range(len(nonTSSfasta)):
        for j in range(45):
            if nonTSSfasta[i][j:j+4]=="TATA":
                TATA_nonTSSfasta.append(nonTSSfasta[i])
    nonTATA_TSSfasta=[]
    nonTATA_nonTSSfasta=[]
    for i in range(len(TSSfasta)):
        if TSSfasta[i] not in TATA_TSSfasta:
            nonTATA_TSSfasta.append(TSSfasta[i])
    for i in range(len(nonTSSfasta)):
        if nonTSSfasta[i] not in TATA_nonTSSfasta:
            nonTATA_nonTSSfasta.append(nonTSSfasta[i])
    print(len(nonTATA_nonTSSfasta))
    TATA_TSSinput=np.array(convert_sequences_to_array(TATA_TSSfasta))
    nonTATA_TSSinput=np.array(convert_sequences_to_array(nonTATA_TSSfasta))
    TATA_nonTSSinput=np.array(convert_sequences_to_array(TATA_nonTSSfasta))
    nonTATA_nonTSSinput=np.array(convert_sequences_to_array(nonTATA_nonTSSfasta))
    return TATA_TSSinput,nonTATA_TSSinput,TATA_nonTSSinput,nonTATA_nonTSSinput

In [14]:
# TATA_TSS,nonTATA_TSS,TATA_nonTSS,nonTATA_nonTSS=\
# get_TSS_nonTSS(gff_name="GCF_000001405.39_GRCh38.p13_genomic.gff",fasta_name="GCF_000001405.39_GRCh38.p13_genomic.fna")

39813

<font size=6> Combine and labels

In [4]:
def TATA_nonTATA_data_labels(TATA_TSS,nonTATA_TSS,TATA_nonTSS,nonTATA_nonTSS):
    TATAinput=list(TATA_TSS)+list(TATA_nonTSS)
    nonTATAinput=list(nonTATA_TSS)+list(nonTATA_nonTSS)
    TATAlabels=[0]*len(TATA_TSS)+[1]*len(TATA_nonTSS)
    TATAlabels=np.array(TATAlabels)
    nonTATAlabels=[0]*len(nonTATA_TSS)+[1]*len(nonTATA_nonTSS)
    nonTATAlabels=np.array(nonTATAlabels)
    TATAinput=np.array(TATAinput)
    nonTATAinput=np.array(nonTATAinput)
    return TATAinput,nonTATAinput,TATAlabels,nonTATAlabels

In [5]:
# TATAinput,nonTATAinput,TATAlabels,nonTATAlabels=\
# TATA_nonTATA_data_labels(TATA_TSS=TATA_TSS,nonTATA_TSS=nonTATA_TSS,TATA_nonTSS=TATA_nonTSS,nonTATA_nonTSS=nonTATA_nonTSS)

<font size=6> Padding

In [6]:
motif_len=21
motif_num=200
pad_array = [0.25,0.25,0.25,0.25]
def pad_sequence_arrays(seq_arrays,pad_array, num, direction):
    pad_array = [0.25,0.25,0.25,0.25]
    padded_seq = []
    for i in seq_arrays:
        if direction == 'both':
            padded_seq.append([pad_array]*num + i.tolist() + [pad_array]*num)
        elif direction == 'pre':
            padded_seq.append([pad_array]*num + i.tolist())
        elif direction == 'post':
            padded_seq.append(i.tolist() + [pad_array]*num)
        else:
            print('Wrong Parameter!!')
    return np.array(padded_seq)

In [7]:
# padded_TATA= pad_sequence_arrays(TATAinput, pad_array, motif_len-1, 'both')
# padded_nonTATA= pad_sequence_arrays(nonTATAinput, pad_array, motif_len-1, 'both')

<font size=6> Pipeline

In [8]:
def main(gff_name,fasta_name):
    TATA_TSS,nonTATA_TSS,TATA_nonTSS,nonTATA_nonTSS=get_TSS_nonTSS(gff_name,fasta_name)
    TATAinput,nonTATAinput,TATAlabels,nonTATAlabels=\
    TATA_nonTATA_data_labels(TATA_TSS=TATA_TSS,nonTATA_TSS=nonTATA_TSS,TATA_nonTSS=TATA_nonTSS,nonTATA_nonTSS=nonTATA_nonTSS)
    padded_TATA= pad_sequence_arrays(TATAinput, pad_array, motif_len-1, 'both')
    padded_nonTATA= pad_sequence_arrays(nonTATAinput, pad_array, motif_len-1, 'both')
    return padded_TATA,padded_nonTATA,TATAlabels,nonTATAlabels


In [22]:
padded_TATA,padded_nonTATA,TATAlabels,nonTATAlabels=\
main(gff_name="GCF_000001405.39_GRCh38.p13_genomic.gff",fasta_name="GCF_000001405.39_GRCh38.p13_genomic.fna")

39775


In [25]:
np.save("T5",padded_TATA)
np.save("nT5",padded_nonTATA)
np.save("T5labels",TATAlabels)
np.save("nT5labels",nonTATAlabels)

In [24]:
len(padded_nonTATA)

59247